In [26]:
from lib import create_mongo_client_to_database_collection

collection_reference = create_mongo_client_to_database_collection('twitter', 'tweets')

In [27]:
collection_reference.count_documents({})

16417

In [28]:
cursor_sampl = collection_reference.aggregate([{'$sample': {'size': 50}}])

In [29]:
list(cursor_sampl)

[{'_id': ObjectId('5b59335935c77d003438b06a'),
  'created_at': 'Thu Jul 26 02:33:42 +0000 2018',
  'id': 1022308950362845184,
  'id_str': '1022308950362845184',
  'text': 'Appreciate you Billy. https://t.co/D8v6zmmbQd',
  'display_text_range': [0, 21],
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  'truncated': False,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 328866492,
   'id_str': '328866492',
   'name': 'Aaron Larsuel',
   'screen_name': 'AaronLarsuel',
   'location': 'Where I be.',
   'url': 'https://itunes.apple.com/us/podcast/americas-lakers-podcast/id1297661311?mt=2',
   'description': 'Host & Producer of America’s Lakers Podcast, official podcast of @Lakers, on @PodcastOne. Big Spoon. IG: @aaronlarsuel  Hit that SUBSCRIBE!!👇🏾',
   'translator_type': 'none',
   'protected': False,
  

| | | | | |
|-|-|-|-|-|
| `_id`        | `truncated`                 |`user`            |`extended_tweet` | `favorited`                   |
| `created_at` | `in_reply_to_status_id`     |`geo`             |`quote_count`    | `retweeted`                   |
| `id`         | `in_reply_to_status_id_str` |`coordinates`     |`reply_count`    | `filter_level`                |
| `id_str`     | `in_reply_to_user_id`       |`place`           |`retweet_count`  | `lang`                        |
| `text`       | `in_reply_to_user_id_str`   |`contributors`    |`favorite_count` | `timestamp_ms`                |
| `source`     | `in_reply_to_screen_name`   |`is_quote_status` |`entities`       |                               |
























![](https://www.evernote.com/l/AAEO7gpKcKdI5YJeGrni4GhdlBiBWdTa3YgB/image.png)

![](https://i.imgflip.com/245tp9.jpg)

## The Aggregation Pipeline

A call to the aggregation framework defines a pipeline (figure 6.1), the **aggregation pipeline**, where the output from each step in the pipeline provides input to the next step. Each step executes a single operation on the input documents to transform the input and generate output documents.

![](https://www.evernote.com/l/AAGxerRxKLZNFrjqxlYK2HPz1R11tr95FFkB/image.png)

### Useful Aggregation Pipeline Operations

- `$project` // Specify fields to be placed in the output document.
- `$match` // Select documents to be processed, similar to find().
- `$limit` // Limit the number of documents to be passed to the next step.
- `$skip` // Skip a specified number of documents.
- `$unwind` // Expand an array, generating one output document for each array entry.
- `$group` // Group documents by a specified key.
- `$sort` // Sort documents.
- `$geoNear` // Select documents near a geospatial location.
- `$out` // Write the results of the pipeline to a collection (new in v2.6).
- `$redact` // Control access to certain data (new in v2.6).

In [6]:
PROJECT = "$project"
MATCH   = "$match"
LIMIT   = "$limit"
UNWIND  = "$unwind"
GROUP   = "$group"
SORT    = "$sort"
COUNT   = "$count"

In [7]:
not_empty = { "$ne" : None }

In [8]:
cursor = collection_reference.aggregate([
    { MATCH : { "geo" : not_empty }},
    { COUNT : "geo"}
])

In [9]:
next(cursor)

{'geo': 1963}

In [12]:
match_non_null_geo = { MATCH : { "geo" : not_empty }}
count_geo = { COUNT : "geo"}

dag_count_non_null_geo = [
    match_non_null_geo,
    count_geo
]

In [13]:
next(collection_reference.aggregate(dag_count_non_null_geo))

{'geo': 1963}

### Group Template

    { $group: { _id: <expression>, <field1>: { <accumulator1> : <expression1> }, ... } }
    
#### Accumulators

- `$sum`
- `$avg`
- `$first`
- `$last`
- `$max`
- `$min`
- `$stdDevPop`
- `$stdDevSamp`

In [14]:
greater_than_10 = { "$gt" : 10 }
sum_1 = { "$sum" : 1 }

def group_and_count(key):
    return { GROUP : {
                 "_id"   : key,
                 "count" : sum_1
                }
           }

match_count_gt_10 = { MATCH : { "count" : greater_than_10 } }

sort_by_count_descending = { SORT : { "count" : -1 } }

def limit(val):
    return { LIMIT : val }

In [15]:
group_and_count("$text")

{'$group': {'_id': '$text', 'count': {'$sum': 1}}}

In [17]:
list(collection_reference.aggregate(
    [
        group_and_count('$lang'),
        match_count_gt_10, 
        sort_by_count_descending,
        limit(5)
    ]
))


[{'_id': 'en', 'count': 13875},
 {'_id': 'und', 'count': 1394},
 {'_id': 'es', 'count': 358},
 {'_id': 'ar', 'count': 115},
 {'_id': 'fr', 'count': 100}]

In [30]:
not_an_empty_array             = { "$ne" : [] }
match_non_empty_hashtag_arrays = { MATCH : { "entities.hashtags" : not_an_empty_array } }
project_to_text_only           = { PROJECT : { "text" : "$entities.hashtags.text", "_id" :0 } }
unwind_text                    = { UNWIND : "$text" }

In [31]:
list(collection_reference.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_only,
        unwind_text,
        limit(10)
    ]
))

[{'text': 'Dodgers'},
 {'text': 'DeadBirds'},
 {'text': 'Dodgers'},
 {'text': 'DeadBirds'},
 {'text': 'thestruggleisreal'},
 {'text': 'actorslife'},
 {'text': 'poolday'},
 {'text': 'heatwave'},
 {'text': 'littlethings'},
 {'text': 'LHHH'}]

In [32]:
list(collection_reference.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_only,
        unwind_text,
        group_and_count('$text'),
        match_count_gt_10, 
        sort_by_count_descending,
        limit(10)
    ]
))

[{'_id': 'job', 'count': 78},
 {'_id': 'Hiring', 'count': 65},
 {'_id': 'LosAngeles', 'count': 64},
 {'_id': 'CareerArc', 'count': 58},
 {'_id': 'earthquake', 'count': 35},
 {'_id': 'Repost', 'count': 31},
 {'_id': 'quake', 'count': 30},
 {'_id': 'california', 'count': 28},
 {'_id': 'hiring', 'count': 26},
 {'_id': 'love', 'count': 25}]

In [35]:
job_hashtags      = ['job', 'jobs', 'hiring', 'careerarc']
location_hashtags = ['california', 'losangeles', 'la', 'santamonica', 'glendale', 'paloalto']
project_to_lower  = { PROJECT : { "text" : {"$toLower" : "$text"} } }
match_not_in_bad  = { MATCH : { "_id" : { "$nin" : job_hashtags + location_hashtags}}}

In [36]:
list(collection_reference.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_only,
        unwind_text,
        project_to_lower,
        group_and_count('$text'),
        match_not_in_bad,
        match_count_gt_10, 
        sort_by_count_descending,
        limit(50)
    ]
))

[{'_id': 'earthquake', 'count': 35},
 {'_id': 'savecolony', 'count': 33},
 {'_id': 'love', 'count': 31},
 {'_id': 'repost', 'count': 31},
 {'_id': 'quake', 'count': 30},
 {'_id': 'dodgers', 'count': 24},
 {'_id': 'queensugar', 'count': 20},
 {'_id': 'wcw', 'count': 19},
 {'_id': 'hollywood', 'count': 15},
 {'_id': 'iexplorechat', 'count': 15},
 {'_id': 'christopherglenn', 'count': 14},
 {'_id': 's2pundercover', 'count': 13},
 {'_id': 'convincesomeonetovotein4words', 'count': 13},
 {'_id': 'maga', 'count': 13},
 {'_id': 'photography', 'count': 13},
 {'_id': 'mixshow', 'count': 12},
 {'_id': 'grindmode', 'count': 12},
 {'_id': 'edm', 'count': 12},
 {'_id': 'trump', 'count': 11},
 {'_id': 'wednesdaywisdom', 'count': 11},
 {'_id': 'outliers', 'count': 11}]